# nanoGPT Development

In [5]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# Read in data, inspect and convert to mapping

with open("input.txt", "r") as infile:
    text = infile.read()
print(f"Dataset length: {len(text)}")

chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(f"Vocab Size: {vocab_size}")

Dataset length: 1115394

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocab Size: 65


# Simple Bigram Model

In [3]:
# Create mapping from characters to integers (character level tokenizer)

STOI = {character: integer for integer, character in enumerate(chars)}
ITOS = {integer: character for integer, character in enumerate(chars)}


def encode(input_: str) -> "list[int]":
    return [STOI[char] for char in input_]


def decode(input_: "list[int]") -> str:
    return "".join([ITOS[integer] for integer in input_])


message = "Hello, there!"
print(f"Encode: {message}: {encode(message)}")
print(f"Decode: Encode({message}): {decode(encode(message))}")

Encode: Hello, there!: [20, 43, 50, 50, 53, 6, 1, 58, 46, 43, 56, 43, 2]
Decode: Encode(Hello, there!): Hello, there!


In [4]:
# Compare the above with OpenAI tiktoken encoding (much larger vocabulary)
import tiktoken

encoding = tiktoken.get_encoding("gpt2")

print(f"tiktoken Vocab Size: {encoding.n_vocab}")
print(f"tiktoken Encode: {encoding.encode(message)}")
print(f"tiktoken Decode: {encoding.decode(encoding.encode(message))}")

tiktoken Vocab Size: 50257
tiktoken Encode: [15496, 11, 612, 0]
tiktoken Decode: Hello, there!


In [5]:
# Convert input data to torch tensor

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [6]:
# Convert the data into a train/test split
n = int(0.9 * len(data))
train_data = data[:n]
test_data = data[n:]

print(f"Num train examples: {train_data.shape}")
print(f"Num test examples: {test_data.shape}")

Num train examples: torch.Size([1003854])
Num test examples: torch.Size([111540])


In [7]:
block_size = 8

train_data[: block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [8]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8


def get_batch(split: str):
    """Generate a mini-batch of training data."""
    if split not in ["train", "test"]:
        raise ValueError("`split` must be one of `train` or `test`.")

    data = train_data if split == "train" else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    return x, y


xb, yb = get_batch("train")

for b in range(batch_size):  # batch dimension
    for t in range(block_size):  # time dimension
        context = xb[b, : t + 1]
        target = yb[b, t]
        print(f"When the input is {context} the target is {target}")
        print(f"{decode(context.tolist())} -> {decode([target.item()])}")

When the input is tensor([24]) the target is 43
L -> e
When the input is tensor([24, 43]) the target is 58
Le -> t
When the input is tensor([24, 43, 58]) the target is 5
Let -> '
When the input is tensor([24, 43, 58,  5]) the target is 57
Let' -> s
When the input is tensor([24, 43, 58,  5, 57]) the target is 1
Let's ->  
When the input is tensor([24, 43, 58,  5, 57,  1]) the target is 46
Let's  -> h
When the input is tensor([24, 43, 58,  5, 57,  1, 46]) the target is 43
Let's h -> e
When the input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target is 39
Let's he -> a
When the input is tensor([44]) the target is 53
f -> o
When the input is tensor([44, 53]) the target is 56
fo -> r
When the input is tensor([44, 53, 56]) the target is 1
for ->  
When the input is tensor([44, 53, 56,  1]) the target is 58
for  -> t
When the input is tensor([44, 53, 56,  1, 58]) the target is 46
for t -> h
When the input is tensor([44, 53, 56,  1, 58, 46]) the target is 39
for th -> a
When the input is 

In [ ]:
torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        """Forward pass. If targets is none, just return the logits."""
        # idx and targets are both (B, T) batches of integers
        logits = self.token_embedding_table(idx)  # (B, T, C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)  # cross entropy expects a (B, C, d...)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        """Sample from the model."""
        # idx is a (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :]  # Becomes (B, C)
            # apply softmax to get the probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

In [21]:
model = BigramLanguageModel(vocab_size)
logits, loss = model(idx=xb, targets=yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.4388, grad_fn=<NllLossBackward0>)


In [29]:
def print_sample(bigram_model, max_new_tokens: int = 100):
    idx = torch.zeros((1, 1), dtype=torch.long)
    print(decode(bigram_model.generate(idx, max_new_tokens=max_new_tokens)[0].tolist()))

In [23]:
# torch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [26]:
batch_size = 32
n_steps = 10_000

for steps in range(n_steps):
    xb, yb = get_batch("train")

    logits, loss = model(idx=xb, targets=yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.405902862548828


In [30]:
print_sample(model, max_new_tokens=300)



G arin gr t id wsse HICon me seave:
D:
Thisthien d cl opoue chtileellenk.
LAngrm
Tha.
ICHe wethareve avered iverefaugiblleato ws,
PAnglland win theve.

Frve cle ad fathigha nobeetonoueay! noonderstiaget nd sss t,
T: y sheethrensetr g tis imaugin:
LI bust ffentr w my mso
Thay tang omyod eros Yor,

A


# Basic Self-Attention Unit

In [8]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32  # batch, time, channels
x = torch.randn(B, T, C)

# Single self-attention head
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x)  # (B, T, head_size)
q = query(x)  # (B, T, head_size)

# Determine the affinities between the tokens
wei = q @ k.transpose(-2, -1)  # (B, T, 16) @ (B, 16, T) -> (B, T, T)

# Aggregation is a data-dependent average of the keys and queries
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])